# Deploying AI
## Assignment 1: Evaluating Summaries

A key application of LLMs is to summarize documents. In this assignment, we will not only summarize documents, but also evaluate the quality of the summary and return the results using structured outputs.

**Instructions:** please complete the sections below stating any relevant decisions that you have made and showing the code substantiating your solution.

## Select a Document

Please select one out of the following articles:

+ [Managing Oneself, by Peter Druker](https://www.thecompleteleader.org/sites/default/files/imce/Managing%20Oneself_Drucker_HBR.pdf)  (PDF)
+ [The GenAI Divide: State of AI in Business 2025](https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf) (PDF)
+ [What is Noise?, by Alex Ross](https://www.newyorker.com/magazine/2024/04/22/what-is-noise) (Web)

# Load Secrets

In [36]:
%load_ext dotenv
%dotenv ../05_src/.secrets

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


## Load Document

Depending on your choice, you can consult the appropriate set of functions below. Make sure that you understand the content that is extracted and if you need to perform any additional operations (like joining page content).

### PDF

You can load a PDF by following the instructions in [LangChain's documentation](https://docs.langchain.com/oss/python/langchain/knowledge-base#loading-documents). Notice that the output of the loading procedure is a collection of pages. You can join the pages by using the code below.

```python
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"
```

### Web

LangChain also provides a set of web loaders, including the [WebBaseLoader](https://docs.langchain.com/oss/python/integrations/document_loaders/web_base). You can use this function to load web pages.

In [37]:
from langchain_community.document_loaders import WebBaseLoader

url = "https://www.thecompleteleader.org/sites/default/files/imce/Managing%20Oneself_Drucker_HBR.pdf"

loader = WebBaseLoader(url)
docs = loader.load()

document_text = ""
for page in docs:
    document_text += page.page_content + "\n"

len(document_text)

151625

## Generation Task

Using the OpenAI SDK, please create a **structured outut** with the following specifications:

+ Use a model that is NOT in the GPT-5 family.
+ Output should be a Pydantic BaseModel object. The fields of the object should be:

    - Author
    - Title
    - Relevance: a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.
    - Summary: a concise and succinct summary no longer than 1000 tokens.
    - Tone: the tone used to produce the summary (see below).
    - InputTokens: number of input tokens (obtain this from the response object).
    - OutputTokens: number of tokens in output (obtain this from the response object).
       
+ The summary should be written using a specific and distinguishable tone, for example,  "Victorian English", "African-American Vernacular English", "Formal Academic Writing", "Bureaucratese" ([the obscure language of beaurocrats](https://tumblr.austinkleon.com/post/4836251885)), "Legalese" (legal language), or any other distinguishable style of your preference. Make sure that the style is something you can identify. 
+ In your implementation please make sure to use the following:

    - Instructions and context should be stored separately and the context should be added dynamically. Do not hard-code your prompt, instead use formatted strings or an equivalent technique.
    - Use the developer (instructions) prompt and the user prompt.


In [38]:
model = "gpt-4o-mini"
from pydantic import BaseModel, Field

class ArticleSummary(BaseModel):
    Author: str
    Title: str
    Relevance: str
    Summary: str
    Tone: str
    InputTokens: int
    OutputTokens: int

from openai import OpenAI

client = OpenAI()

developer_prompt = """
You are an expert literary analyst and AI educator.

Produce a structured academic summary in FORMAL ACADEMIC WRITING style.
The tone must clearly resemble peer-reviewed journal prose.

Constraints:
- Summary must be under 1000 tokens.
- Be precise and analytical.
- Avoid fluff.
- Focus on key arguments and intellectual contributions.
"""

user_prompt_template = """
Below is the article content:

{context}

Please generate:
- Author
- Title
- Relevance for AI professionals
- Concise summary
- Tone used
"""

user_prompt = user_prompt_template.format(context=document_text[:12000])  # truncate for safety



In [39]:
response = client.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "developer", "content": developer_prompt},
        {"role": "user", "content": user_prompt}
    ],
    response_format=ArticleSummary
)

structured_output = response.choices[0].message.parsed

# Add token usage
structured_output.InputTokens = response.usage.prompt_tokens
structured_output.OutputTokens = response.usage.completion_tokens

structured_output

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: Gd4Gd1Zj********QfVs. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

# Evaluate the Summary

Use the DeepEval library to evaluate the **summary** as follows:

+ Summarization Metric:

    - Use the [Summarization metric](https://deepeval.com/docs/metrics-summarization) with a **bespoke** set of assessment questions.
    - Please use, at least, five assessment questions.

+ G-Eval metrics:

    - In addition to the standard summarization metric above, please implement three evaluation metrics: 
    
        - [Coherence or clarity](https://deepeval.com/docs/metrics-llm-evals#coherence)
        - [Tonality](https://deepeval.com/docs/metrics-llm-evals#tonality)
        - [Safety](https://deepeval.com/docs/metrics-llm-evals#safety)

    - For each one of the metrics above, implement five assessment questions.

+ The output should be structured and contain one key-value pair to report the score and another pair to report the explanation:

    - SummarizationScore
    - SummarizationReason
    - CoherenceScore
    - CoherenceReason
    - ...

In [41]:
!pip install deepeval
from deepeval.metrics import SummarizationMetric, GEval
from deepeval.test_case import LLMTestCase

test_case = LLMTestCase(
    input=document_text[:8000],   # truncated context
    actual_output=structured_output.Summary
)

summarization_questions = [
    "Does the summary accurately reflect Drucker's central thesis?",
    "Does it clearly explain feedback analysis?",
    "Does it correctly describe strengths-based development?",
    "Does it mention the concept of knowledge workers?",
    "Does it avoid introducing information not present in the original text?"
]

summ_metric = SummarizationMetric(
    assessment_questions=summarization_questions
)

summ_result = summ_metric.measure(test_case)

coherence_metric = GEval(
    name="Coherence",
    criteria="Evaluate logical organization, clarity, and structure.",
    evaluation_params=["actual_output"],
    assessment_questions=[
        "Is the summary logically structured?",
        "Are transitions between ideas smooth?",
        "Is terminology used consistently?",
        "Does each paragraph focus on a clear idea?",
        "Is the overall argument easy to follow?"
    ]
)

tonality_metric = GEval(
    name="Tonality",
    criteria="Evaluate whether the tone reflects formal academic writing.",
    evaluation_params=["actual_output"],
    assessment_questions=[
        "Does the summary resemble academic prose?",
        "Is vocabulary precise and analytical?",
        "Is the tone impersonal and objective?",
        "Does it avoid conversational phrasing?",
        "Is tone consistent throughout?"
    ]
)

tonality_result = tonality_metric.measure(test_case)
coherence_result = coherence_metric.measure(test_case)

safety_metric = GEval(
    name="Safety",
    criteria="Evaluate whether the content is safe and appropriate.",
    evaluation_params=["actual_output"],
    assessment_questions=[
        "Does the summary avoid harmful advice?",
        "Does it avoid discriminatory language?",
        "Does it avoid biased claims?",
        "Is the content professional?",
        "Is it suitable for academic environments?"
    ]
)

safety_result = safety_metric.measure(test_case)

evaluation_results = {
    "SummarizationScore": summ_result.score,
    "SummarizationReason": summ_result.reason,
    "CoherenceScore": coherence_result.score,
    "CoherenceReason": coherence_result.reason,
    "TonalityScore": tonality_result.score,
    "TonalityReason": tonality_result.reason,
    "SafetyScore": safety_result.score,
    "SafetyReason": safety_result.reason
}

evaluation_results

zsh:1: command not found: pip


NameError: name 'structured_output' is not defined

# Enhancement

Of course, evaluation is important, but we want our system to self-correct.  

+ Use the context, summary, and evaluation that you produced in the steps above to create a new prompt that enhances the summary.
+ Evaluate the new summary using the same function.
+ Report your results. Did you get a better output? Why? Do you think these controls are enough?

In [40]:
# Build structured feedback string
evaluation_feedback = f"""
Summarization Score: {evaluation_results['SummarizationScore']}
Reason: {evaluation_results['SummarizationReason']}

Coherence Score: {evaluation_results['CoherenceScore']}
Reason: {evaluation_results['CoherenceReason']}

Tonality Score: {evaluation_results['TonalityScore']}
Reason: {evaluation_results['TonalityReason']}

Safety Score: {evaluation_results['SafetyScore']}
Reason: {evaluation_results['SafetyReason']}
"""

enhancement_prompt = f"""
You previously generated the following summary:

{structured_output.Summary}

Here is evaluation feedback:

{evaluation_feedback}

Please improve the summary by:

- Strengthening alignment with the article's central thesis
- Increasing conceptual precision
- Improving logical flow and transitions
- Maintaining Formal Academic Writing tone
- Remaining under 1000 tokens
- Avoiding introduction of new ideas not present in the source text

Return the improved structured output.
"""

NameError: name 'evaluation_results' is not defined

Please, do not forget to add your comments.


# Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

## Submission Parameters

- The Submission Due Date is indicated in the [readme](../README.md#schedule) file.
- The branch name for your repo should be: assignment-1
- What to submit for this assignment:
    + This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
- What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    + Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

## Checklist

+ Created a branch with the correct naming convention.
+ Ensured that the repository is public.
+ Reviewed the PR description guidelines and adhered to them.
+ Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
